<figure style="float:right">
<a href="http://c51.lbl.gov/~walkloud/callat/">
    <img
    src="./data/callat_logo.png"
    width="150"
    alt="CalLat logo"
    /img>
</a>
</figure>

# Jupyter notebook for CalLat gA project 

## Import libraries

In [ ]:
from __future__ import print_function

import pandas as pd
pd.options.display.max_rows = 16
import numpy as np
import scipy as sp
%matplotlib inline
import matplotlib as mpl
import lsqfit
import gvar as gv
import callat_ga_lib as xlib
import sys
print("python version:", sys.version)
print("pandas version:", pd.__version__)
print("numpy  version:", np.__version__)
print("scipy  version:", sp.__version__)
print("mpl    version:", mpl.__version__)
print("lsqfit version:", lsqfit.__version__)
print("gvar   version:", gv.__version__)

## Define analysis parameters
* `switches['ensembles']` | list of strings
    * select the ensembles that are used to perform the extrapolation
    * the three rows correspond to the 0.15, 0.12, and 0.09 fm ensembles that are available
* `switches['ansatz']` | dictionary
    * define the fit ansatz for the extrapolation
    * `['type']` | string: chooses between a Taylor expansion or Baryon Xpt
        * Taylor expansion only includes even powers of ε<sub>π</sub>
    * `['truncation']` | integer: is an integer n corresponding to the order of ε<sub>π</sub><sup>n</sup>
    * `['FV']` | boolean: True turns on NLO FV corrections for both Baryon Xpt and Taylor
    * `['xsb']` | boolean: True turns on am<sub>π</sub> term for Baryon Xpt
    * `['alpha']` | boolean: True turns on α<sub>s</sub>a<sup>2</sup> for Baryon Xpt

In [ ]:
switches = dict()
# Ensembles used in extrapolation
switches['ensembles'] = [
    'a15m400','a12m400','a09m400',
    'a15m350','a12m350','a09m350',
    'a15m310','a12m310','a09m310',
    'a15m220','a12m220','a09m220','a12m220S','a12m220L',
    'a15m130','a12m130'
    ]
switches['x_shift'] = {
    'a15m400':0,    'a12m400':0,  'a09m400':0,
    'a15m350':0,    'a12m350':0,  'a09m350':0,
    'a15m310':0,    'a12m310':0,  'a09m310':0,
    'a15m220':-.002,'a12m220':.00,'a09m220':.004,'a12m220S':-.003,'a12m220L':.002,
    'a15m130':-.003,'a12m130':.001
    }

switches['ansatz'] = dict()
### Type of fit: 'xpt_N', 'taylor_N', 'linear_N', 'constant_N', 'xpt-full_4', 'xpt-doublelog_4', 'xpt-delta_N'
switches['ansatz']['type'] = ['xpt_3','xpt_4','taylor_2','taylor_4','linear_2','linear_4']
#switches['ansatz']['type'] = ['xpt-delta_3']
switches['ansatz']['FV'] = True # True turns on NLO FV correction
switches['ansatz']['FVn'] = 3 # FV(epi^n) where n in [0,2,3]
switches['ansatz']['xsb'] = False # True turns on O(a) discretization
switches['ansatz']['alpha'] = False # True turns on O(alpha_s a^2) discretization

### NOTEBOOK Report
switches['report'] = dict()
switches['report']['print_fit'] = False
switches['report']['lecs'] = True #print LECs
switches['report']['lecs_full'] = False #True print ALL LECs, False, print from lec list
switches['report']['correlation'] = True

### Save figs to local directory?
switches['save_figs'] = True
### plot tools 
switches['plot'] = dict()
switches['plot']['raw_data'] = False
switches['plot']['chiral'] = True
switches['plot']['continuum'] = False
switches['plot']['FV'] = False
switches['plot']['model_avg_histogram'] = False
### For the switches below to work
### the corresponding plot above must be True
switches['plot']['model_avg_chiral'] = True
switches['plot']['model_avg_cont'] = False
switches['plot']['model_avg_fv'] = False

### multiplicative factor for prior width of epsilon_delta
switches['eps_delta_sig'] = 0.05
switches['axial_sig'] = 0.18 # 0.18 the optimal width based upon NLO fits and Bayes factors
### inflation of prior widths
### multiplicative factor for NxLO LECs
p_lo = 1.
p_nlo = 1.
p_nnlo = 1.
p_nnloa = 1.
### prior width scaling for epi**3 FV function
fv_3_width = 1.0
### FV coefficient in Taylor expansion analyses
g0_fv = 1.2
g0_fv_width = 1. # loose-to-tight [1.0, 0.3, 0.1]
### remove estimated QED corrections from mpi and Fpi
switches['qed'] = False
isospin = 0.00038 # Half the difference between analysis with and without QED corrections

## Define priors and PDG values
`gvar` datatype has the form `gv.gvar(mean, standard deviation)`

[gvar documentation](https://github.com/gplepage/gvar/blob/master/doc/gvar.pdf)

In [ ]:
priors = dict()
# Xpt priors
priors['xpt'] = dict()
priors['xpt']['g0'] = gv.gvar(1.0, p_lo*50.0) # LO LEC
priors['xpt']['a1'] = gv.gvar(0.0, 1E-3) # DWF order a discretization
priors['xpt']['c2'] = gv.gvar(0.0, p_nlo*50.0) # NLO counterterm epi^2
priors['xpt']['c3'] = gv.gvar(0.0, p_nnlo*50.0) # NNLO LEC epi^3
priors['xpt']['a2'] = gv.gvar(0.0, p_nlo*50.0) # NLO a^2
priors['xpt']['s2'] = gv.gvar(0.0, 1.0) # NLO alpha_s a^2
priors['xpt']['a4'] = gv.gvar(0.0, p_nnloa*p_nnlo*1.0) # NNNLO a^4
priors['xpt']['b4'] = gv.gvar(0.0, p_nnlo*1.0) # NNNLO a^2 epi^2
priors['xpt']['c4'] = gv.gvar(0.0, p_nnlo*1.0) # NNNLO epi^4
priors['xpt']['gm4'] = gv.gvar(0.0, 50.0) # NNNLO log term
priors['xpt']['gnd0'] = gv.gvar(-6./5*1.2,switches['axial_sig']*6/5*1.2) # delta LECs
priors['xpt']['gdd0'] = gv.gvar(-9./5*1.2,switches['axial_sig']*9/5*1.2) # delta LECs
priors['xpt']['f3'] = gv.gvar(0.,fv_3_width*23.0) # epi^3 FV coefficient # delta is 46.0 else 23.0
# taylor priors
priors['taylor'] = dict()
priors['taylor']['g0'] = gv.gvar(g0_fv, p_nlo*g0_fv_width) # FV coefficient
priors['taylor']['c0'] = gv.gvar(1.0, p_lo*50.0) # constant
priors['taylor']['c2'] = gv.gvar(0.0, p_nlo*50.0) # epi^2
priors['taylor']['a2'] = gv.gvar(0.0, p_nlo*50.0) # a^2
priors['taylor']['c4'] = gv.gvar(0.0, p_nnlo*1.0) # epi^4
priors['taylor']['a4'] = gv.gvar(0.0, p_nnloa*p_nnlo*1.0) # a^4
priors['taylor']['b4'] = gv.gvar(0.0, p_nnlo*1.0) # a^2 epi^2
priors['taylor']['f3'] = gv.gvar(0.,fv_3_width*18.0) # 18 epi^3 FV coefficient
# linear priors
priors['linear'] = dict()
priors['linear']['g0'] = gv.gvar(g0_fv, p_nlo*g0_fv_width) # FV coefficient
priors['linear']['c0'] = gv.gvar(1.0, p_lo*50.0) # constant
priors['linear']['c2'] = gv.gvar(0.0, p_nlo*50.0) # epi
priors['linear']['a2'] = gv.gvar(0.0, p_nlo*50.0) # a^2
priors['linear']['a4'] = gv.gvar(0.0, p_nnloa*p_nnlo*1.0) # a^4
priors['linear']['c4'] = gv.gvar(0.0, p_nnlo*1.0) # epi^2
priors['linear']['f3'] = gv.gvar(0.,fv_3_width*12.5) # epi^3 FV coefficient
# constant priors
priors['constant'] = dict()
priors['constant']['g0'] = gv.gvar(g0_fv, p_nlo*g0_fv_width) # FV coefficient
priors['constant']['c0'] = gv.gvar(1.0, p_lo*10.0) # constant
priors['constant']['a2'] = gv.gvar(0.0, p_nlo*10.0) # a^2
priors['constant']['a4'] = gv.gvar(0.0, p_nnloa*p_nnlo*1.0) # a^4
priors['constant']['f3'] = gv.gvar(0.,fv_3_width) # epi^3 FV coefficient


# Physical parameters from PDG
phys_params = dict()
# http://pdg.lbl.gov/2016/tables/rpp2016-tab-mesons-light.pdf
phys_params['mpi'] = gv.gvar(139.57018, 0.00035) # mpi +/- [MeV]
# http://pdg.lbl.gov/2016/reviews/rpp2016-rev-pseudoscalar-meson-decay-cons.pdf
phys_params['fpi'] = gv.gvar(130.2, 1.7) # fpi + ['MeV']
# Turn off QED?
if switches['qed']:
    phys_params['mpi'] = gv.gvar(134.9770, 0.0005) # mpi 0 [MeV]
    phys_params['fpi'] = gv.gvar(130.2, 1.7)/(1+0.5*gv.gvar(0.0169,.0015))
# http://pdg.lbl.gov/2017/listings/rpp2017-list-Delta-1232.pdf
phys_params['Delta'] = gv.gvar(293, 2) # Delta(1232) Breit Wigner Mass

## Import data
[pandas dataframe documentation](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html)

In [ ]:
# import correlator bootstraps
gadf = pd.read_csv('./data/github_ga_v2.csv')
gadf.groupby('ensemble').describe()[['ga','epi','mpil']]

In [ ]:
# import HISQ parameters
hqdf = pd.read_csv('./data/hisq_params.csv')
hqdf

## Format data for analysis

In [ ]:
data = xlib.format_data(switches, gadf, hqdf)

## Chiral-continuum fit
[lsqfit documentation](https://github.com/gplepage/lsqfit/blob/master/doc/lsqfit.pdf)

In [ ]:
result = xlib.fit_data(switches, priors, data, phys_params)
lecs = {
    'xpt_2'      :['g0','c2'],
    'xpt_3'      :['g0','c2','c3'],
    'xpt_4'      :['g0','c2','c3','c4'],
    'xpt-full_4' :['g0','c2','c3','c4','gm4'],
    'xpt-delta_2':['g0','c2','gnd0','gdd0'],
    'xpt-delta_3':['g0','c2','c3','gnd0','gdd0'],
    'xpt-delta_4':['g0','c2','c3','c4','gnd0','gdd0'],
    'taylor_2'   :['c0','c2','g0'],
    'taylor_4'   :['c0','c2','c4','g0'],
    'linear_2'   :['c0','c2','g0'],
    'linear_4'   :['c0','c2','c4','g0'],
}
for a in switches['ansatz']['type']:
    #print(result[a]['fit']) #uncomment to print entire fit results
    print("\n%s physical point result:" %a, result[a]['phys']['result'])
    print('%s, %s, %s, %s' %(result[a]['phys']['result'].mean, result[a]['phys']['result'].sdev, result[a]['fit'].chi2/result[a]['fit'].dof, result[a]['fit'].logGBF))
    print(result[a]['fit'].dof)
    if switches['report']['lecs']:
        print('order-by-order contributions')
        sub = result[a]['phys']['order'][0]
        print('1: %s & %s' %(sub,100*sub/result[a]['phys']['result'].mean))
        for idx,t in enumerate(result[a]['phys']['order'][1:]):
            print('%s: %s & %s' %(2+idx,t-sub,100*(t-sub)/result[a]['phys']['result'].mean))
            sub = t
        print('LEC correlation matrix')
        key_list = []
        p_list = []
        for k in result[a]['fit'].p.keys():
            if switches['report']['lecs_full']:
                if a in k:
                    key_list.append(k)
                    p_list.append(result[a]['fit'].p[k])
            else:
                if (a in k) and (k.split('_')[-1] in lecs[a]):
                    key_list.append(k)
                    p_list.append(result[a]['fit'].p[k])
        # put LECS in human-sensible order
        if not switches['report']['lecs_full']:
            key_list2 = []
            for k in lecs[a]:
                for kn in key_list:
                    if k in kn:
                        key_list2.append(kn)
        key_list = key_list2
        pcorr = gv.evalcorr(p_list)
        for idx1,k in enumerate(key_list):
            string = ''
            for idx2,l in enumerate(key_list):
                string += '%.5f & ' %pcorr[idx1,idx2]
            print(string,k,result[a]['fit'].p[k])
    if switches['report']['print_fit']:
        print(result[a]['fit'].format(maxline=True,pstyle='m'))

## Systematic error budget
The total uncertainty here differs from the paper because uncertainty from finite volume and isospin breaking have yet to be included. These two systematic uncertainties are estimated independently and added in quadrature after this analysis.

In the paper, the input uncertainty is absorbed into the statistical uncertainty.

In [ ]:
error = xlib.error_budget(switches,result)
for a in switches['ansatz']['type']:
    print(a)
    print(pd.DataFrame.from_dict(error[a]['pct']).rename(index={0: 'pct. uncertainty'})[['stat','chiral','disc','fv','total']])
    print('')

In [ ]:
res_list = dict()
for a in switches['ansatz']['type']:
    #print(a)
    #print(error[a]['std'])
    res_list[a] = result[a]['phys']['result']

## Plot results
* chiral extrapolation
* convergence of extrapolation
* continuum extrapolation
* infinite volume extrapolation

In [ ]:
# Chiral extrapolation and series convergence
if switches['plot']['chiral']:
    Plot = xlib.plot_chiral_fit()
    r_chiral, r_converge = Plot.plot_chiral(switches,data,result)
    mpl.pyplot.show()

In [ ]:
# Continuum extrapolation
if switches['plot']['continuum']:
    Plot = xlib.plot_chiral_fit()
    r_cont = Plot.plot_continuum(switches,data,result)
    mpl.pyplot.show()

In [ ]:
# Infinite volume extrapolation at 220 MeV 0.12 fm
if switches['plot']['FV']:
    Plot = xlib.plot_chiral_fit()
    r_fv = Plot.plot_volume(switches,data,result)
    mpl.pyplot.show()

## Model averaging
Model average with Bayes Factors provides more robust prediction than choosing any single result.

From marginalizing over models $M_k$ for $k \in \{\textrm{models}\}$ we get the averaged posterior distribution to be:

$P(g_A | \textrm{data}) = \sum_k P(g_A | M_k, \textrm{data})P(M_k|\textrm{data})$

where

$P(M_i|\textrm{data}) = \frac{\textrm{BF}_i P(M_i)}{\sum_k \textrm{BF}_k P(M_k)} = \frac{\textrm{BF}_i}{\sum_k \textrm{BF}_k}$

where the second equality is true if there are no a priori preferred models.

In [ ]:
# extra uncertainties in quadrature
error, plot_params = xlib.bma(switches,result,isospin)
print('P(Mk|D) & result & model')
for k in error['weights'].keys():
    print('%.3f& %s & %s' %(error['weights'][k],str(result[k]['phys']['result']),k))
#print('\nmodel correlations')
#mcorr = gv.evalcorr(error['gA_dict'])
#for s in switches['ansatz']['type']:
#    string = ''
#    for t in switches['ansatz']['type']:
#        string += '%.3f & ' %mcorr[(s,t)]
#    print(string,s)
print('\ngA = %f +- %f +- %f' %(error['E(gA)'],error['s(gA)'],error['s(Mk)']))
print ('percent uncertainty: %f +- %f' %(error['s(gA)']/error['E(gA)']*100,error['s(Mk)']/error['E(gA)']*100))
print('\ngA = %f, %f' %(error['E(gA)'],np.sqrt(error['s(gA)']**2+error['s(Mk)']**2)))

print('percent uncertainty: %f' %(np.sqrt(error['s(gA)']**2+error['s(Mk)']**2)/error['E(gA)']*100))
print("\nError budget from extrapolation")
print(pd.DataFrame.from_dict(error['pct_budget']).rename(index={0: 'pct. err'})[['stat','chiral','disc','fv','isospin','model','total']].round(3))

In [ ]:
error

## Correlation Between physical point and different pion masses

- Use the resulting fits to compute the correlation between $g_A(m_\pi^{phys})$ and $g_A(m_\pi)$
- Use this correlation to cmopute the shift in the physical point prediction due to a 1-$\sigma$ fluctuation at the different pion mass points used in the calculation

In [ ]:
if switches['report']['correlation']:
    xlib.mpi_corr(switches,phys_params,result,error)

In [ ]:
if switches['plot']['model_avg_histogram']:
    Plot = xlib.plot_chiral_fit()
    Plot.plot_histogram(switches,plot_params)

In [ ]:
if switches['plot']['model_avg_chiral'] and switches['plot']['chiral']:
    Plot = xlib.plot_chiral_fit()
    Plot.model_avg_chiral(switches,phys_params,error['weights'],r_chiral,data)

In [ ]:
if switches['plot']['model_avg_cont'] and switches['plot']['continuum']:
    Plot = xlib.plot_chiral_fit()
    Plot.model_avg_cont(switches,error['weights'],r_cont)

In [ ]:
if switches['plot']['model_avg_fv'] and switches['plot']['FV']:
    Plot = xlib.plot_chiral_fit()
    Plot.model_avg_fv(switches,error['weights'],r_fv)

<center>
    <span style="color: black; font-family: Helvetica; font-size: 2em">
        These calculations are made possible by
    </span>
</center>

| | |
|:---:|:---:|
| [<img src='./data/incite_logo.png' width='200'/>](http://www.doeleadershipcomputing.org/)  | [<img src='./data/olcf_logo.png' width='320'/>](https://www.olcf.ornl.gov/) |
| [<img src='./data/llnl_logo.png' width='640' />](https://hpc.llnl.gov/) | [<img src='./data/scidac_logo.png' width='350' />](http://www.scidac.gov/) |